In [1]:
import spacy
from langdetect import detect
import json

In [9]:
from utils import join_tokens

In [10]:
def language_detector(tokenized_text = list[str],num_tokens = 50) -> str:
    if len(tokenized_text) < num_tokens: 
        text = join_tokens(tokenized_text)
    else:
        text = join_tokens(tokenized_text[:num_tokens])
    
    return detect(text)

In [23]:
with open(f"data/ner/complete_ner_data.json",'r') as f:
    data = json.load(f)

In [24]:
ex = data[12094]
print(ex['ner'])

[[47, 48, 'person'], [119, 120, 'person'], [275, 276, 'person'], [337, 338, 'person'], [434, 435, 'person']]


In [25]:
print(language_detector(ex['tokenized_text']))
print(ex['tokenized_text'])

fr
['Bundesgericht', 'Tribunal', 'fédéral', 'Tribunale', 'federale', 'Tribunal', 'federal', '{', 'T', '0', '/', '2', '}', '6B_632', '/', '2008', '/', 'bri', 'Arrêt', 'du', '10', 'mars', '2009', 'Cour', 'de', 'droit', 'pénal', 'Composition', 'MM', '.', 'les', 'Juges', 'Favre', ',', 'Président', ',', 'Schneider', 'et', 'Mathys', '.', 'Greffier', ':', 'M', '.', 'Oulevey', '.', 'Parties', 'Patrick', 'Ortiz', ',', 'recourant', ',', 'représenté', 'par', 'Me', 'François', 'Canonica', ',', 'avocat', ',', 'contre', 'Procureur', 'général', 'du', 'canton', 'de', 'Genève', ',', 'case', 'postale', '3565', ',', '1211', 'Genève', '3', ',', 'intimé', '.', 'Objet', 'Infraction', 'grave', 'à', 'la', 'loi', 'fédérale', 'sur', 'les', 'stupéfiants', ',', 'recours', 'contre', 'l', "'", 'arrêt', 'de', 'la', 'Cour', 'de', 'justice', 'du', 'canton', 'de', 'Genève', ',', 'Chambre', 'pénale', ',', 'du', '23', 'juin', '2008', '.', 'Faits', ':', 'A', '.', 'En', '2007', ',', 'Patrick', 'Ortiz', 'a', 'vendu', 'cinqu

# language-based sentencer splitter

In [5]:
#initialize models

nlp_de = spacy.load("de_core_news_sm")
nlp_fr = spacy.load("fr_core_news_sm")
nlp_it = spacy.load("it_core_news_sm")

nlp_models = {"de" : nlp_de,
            "fr" : nlp_fr,
            "it" : nlp_it}

In [6]:
def split_into_sentences(text, language_code,default = 'de'):
    """Split the given text into sentences based on the language."""
    nlp = nlp_models.get(language_code,nlp_models['de'])
    doc = nlp(text)
    return [sent.text for sent in doc.sents]



In [14]:
split_into_sentences(join_tokens(ex['tokenized_text']),'fr')

['Participants à la procédure Ernst Hagemann, représenté par Me Isabelle Jaques, avocate, recourant, contre Ministère public central du canton de Vaud, avenue de Longemalle 1, 1020',
 'Renens VD, intimé.',
 'Objet',
 'Incendie intentionnel;',
 "mesure institutionnelle en milieu fermé; arbitraire, présomption d ' innocence, droit d ' être entendu, recours contre l ' arrêt de la Cour d ' appel pénale du Tribunal cantonal du canton de Vaud du 20 février 2015.",
 'Faits:',
 'A.',
 'Par jugement du 16 septembre 2014',
 ", le Tribunal criminel de l ' arrondissement de l ' Est vaudois a reconnu Ernst Hagemann coupable de mise en danger de la vie d ' autrui, vol, tentative de vol, menaces, incendie intentionnel et tentative d ' instigation à faux témoignage en justice.",
 "Le prévenu a été condamné à une peine privative de liberté de six ans et une mesure d ' internement au sens de l ' art.",
 '64 CP a été ordonnée.',
 "Statuant le 20 février 2015 sur appel et appel joint du prévenu, respectiv

# some utils

In [17]:
def sort_and_clean_ner_list(ner : list[list]) -> list[list]:
    ner = sorted(ner,key = lambda x : x[0]) #sort on the start_token_index in increasing order

    #remove overlapping ner occurences
    last_end = -1
    output_ner = []

    for ner_instance in ner:
        start = ner_instance[0]
        if start > last_end:
            output_ner.append(ner_instance)
            last_end = ner_instance[1]
    
    return output_ner

In [21]:
#do some testing

for ex in data[10:20]:
    print(sort_and_clean_ner_list(ex['ner']))
    print(len(sort_and_clean_ner_list(ex['ner'])))
    print(len(ex['ner']))
    print('\n')

[[4, 5, 'person'], [86, 87, 'person'], [123, 124, 'person'], [179, 180, 'person']]
4
4


[]
0
0


[[32, 33, 'person'], [34, 35, 'person'], [39, 40, 'person'], [41, 42, 'person'], [50, 51, 'person'], [119, 119, 'location'], [129, 130, 'person'], [131, 132, 'person'], [141, 142, 'person'], [168, 169, 'person'], [206, 207, 'person'], [224, 225, 'person']]
12
12


[]
0
0


[[4, 5, 'person'], [10, 12, 'organization']]
2
2


[]
0
0


[]
0
0


[[4, 5, 'person'], [138, 139, 'person'], [206, 207, 'person'], [302, 303, 'person']]
4
4


[[4, 5, 'person'], [19, 20, 'person'], [58, 59, 'person'], [178, 179, 'person'], [223, 224, 'person'], [228, 229, 'person'], [238, 239, 'person'], [250, 251, 'person'], [855, 856, 'person']]
9
9


[[436, 437, 'person'], [1464, 1465, 'person']]
2
2




def 